In [0]:
dbutils.widgets.text("SinkPath", "", "Enter Sink Path")
sinkPath = dbutils.widgets.get("SinkPath")
print(sinkPath)

In [0]:
from pyspark.sql.functions import col, expr, from_json, DataFrame, sha2, concat_ws, lit
from pyspark.sql.types import *

In [0]:
# Configuration for Azure Blob Storage SAS token
storage_account = "k1datastorage"
container = "casestudy"
sas_token = "sp=racwdlmeop&st=2025-08-09T09:56:09Z&se=2025-08-09T18:11:09Z&spr=https&sv=2024-11-04&sr=c&sig=BK5xy4gjRsv7KoX1NqmyOJt5tQQeAAszSDQMg8l3yh4%3D"

# Set SAS token for Azure Blob Storage container
spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net", sas_token)

# Specify the directory or folder path in your container
# sinkPath = "bronzelayer"  # You can modify this to your actual folder path
# List files in the container (without specifying folder path) to check the structure

dbutils.fs.ls(f"wasbs://{container}@{storage_account}.blob.core.windows.net/")
print('-----------------------------------------------------------------------------------')
# Delta path to be accessed
delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/raw/bronze"

# List the files in the specified path
files = dbutils.fs.ls(delta_path)

# Print out the contents of the path
for file in files:
    print(f"File name: {file.name}, File path: {file.path}")

-----------------------------------------------------------------------------------
File name: Customer/, File path: wasbs://casestudy@k1datastorage.blob.core.windows.net/raw/bronze/Customer/
File name: Products/, File path: wasbs://casestudy@k1datastorage.blob.core.windows.net/raw/bronze/Products/
File name: Transactions/, File path: wasbs://casestudy@k1datastorage.blob.core.windows.net/raw/bronze/Transactions/


In [0]:
cust_path =f"{delta_path}/Customer"
trans_path =f"{delta_path}/Transactions"
pro_path =f"{delta_path}/Products"

cust_df = spark.read.csv(cust_path)
trans_df = spark.read.csv(trans_path)
pro_df = spark.read.csv(pro_path)
cust_df.printSchema()
trans_df.printSchema()
pro_df.printSchema()

root
 |-- _c0: string (nullable = true)

root
 |-- _c0: string (nullable = true)

root
 |-- _c0: string (nullable = true)



In [0]:
no_dup_cust_df = cust_df.dropDuplicates()
no_dup_trans_df = trans_df.dropDuplicates()
no_dup_pro_df = pro_df.dropDuplicates()

silver_path = f"{delta_path}Silver"

no_dup_cust_df.write.mode("overwrite").format("delta").save(silver_path)
no_dup_trans_df.write.mode("overwrite").format("delta").save(silver_path)
no_dup_pro_df.write.mode("overwrite").format("delta").save(silver_path)

In [0]:
cust_silver = spark.read.format("delta").load(silver_path)
trans_silver = spark.read.format("delta").load(silver_path)
pro_silver = spark.read.format("delta").load(silver_path)

def mask_pii(df: DataFrame, pii_columns: dict) -> DataFrame:
    columns = df.columns

    for index, mask_type in pii_columns.items():
        if index >= len(columns):
            continue  

        col_name = columns[index]
        if col_name not in df.columns:
            continue

        if mask_type == "hash":
            df = df.withColumn(col_name, sha2(col(col_name).cast("string"), 256))
        elif mask_type == "redact":
            df = df.withColumn(col_name, lit("REDACTED"))
        else:
            raise ValueError(f"Unsupported mask type: '{mask_type}' for column '{col_name}'")

    return df

pii_columns = {
    1: "hash",   
    2: "hash",  
    3: "redact", 
    4: "redact"
}

masked_customers_df = mask_pii(cust_silver, pii_columns)
masked_customers_df.show(5)

gold_path = f"{delta_path}Gold"

masked_customers_df.write.mode("overwrite").format("delta").save(gold_path)

+--------------------+
|                 _c0|
+--------------------+
|                $|
| \b    ...|
|�\a. &�\f...|
|           <  \f|
|�\a &�5...|
+--------------------+
only showing top 5 rows
